# Movement of actors

ABSESpy provides methods such as `move_to`, `put_on`, `put_on_layer` at the agent level to reposition agents across the grid or grids layers that support the model. Model agents' ability to explore their environment is crucial in the context of agent-based simulation and complex systems.

In [1]:
from abses import Actor, MainModel
from abses.datacollection import DataCollector


class MovingActor(Actor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def step(self):
        # pick a random, neighboring cell or position to move to
        neighborhood = self.layer.get_neighborhood(self.pos, moore=True)

        new_pos = self.random.choice(neighborhood)
        self.move_to(new_pos)


class MockModel(MainModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def setup(self):
        # @TODO: create another layer, a rasterized one, to illustrate movement

        self.agents.create(MovingActor, 10)

        self.nature.create_module(how="from_resolution", shape=(10, 10))

        for actor in self.actors:
            actor.put_on_layer(layer=self.nature.major_layer, pos=(0, 0))

        self.datacollector = DataCollector(
            self, agent_reporters={"position": "pos"}
        )

    def step(self):
        # step agents
        self.actors.trigger("step")

        # collect data
        self.datacollector.collect()

In [2]:
model = MockModel(seed=42, parameters={"time": {"end": 10}})

model.run_model()

[2024-01-07 14:26:13][human               ] | Initializing a new Human Module...
[2024-01-07 14:26:13][nature              ] | Initializing a new Base Nature module...
[2024-01-07 14:26:13][main                ] | Setting up MockModel...
[2024-01-07 14:26:13][container           ] | Created 10 actors of breed MovingActor
[2024-01-07 14:26:13][nature              ] | Initializing a new Model Layer...
[2024-01-07 14:26:13][nature              ] | Using rioxarray version: 0.13.4
[2024-01-07 14:26:13][main                ] | Ending MockModel


In [3]:
model.datacollector.get_agent_vars_dataframe().unstack(level=-1)

position                                                          \
AgentID       1       2       3       4       5       6       7       8    
Step                                                                       
0         (0, 1)  (1, 1)  (1, 1)  (0, 1)  (0, 1)  (0, 1)  (1, 0)  (0, 1)   
1         (0, 0)  (0, 0)  (0, 0)  (1, 2)  (0, 2)  (1, 1)  (0, 0)  (1, 2)   
2         (1, 0)  (1, 1)  (1, 0)  (2, 2)  (0, 1)  (1, 0)  (1, 1)  (2, 2)   
3         (0, 0)  (0, 1)  (0, 1)  (3, 1)  (0, 0)  (0, 1)  (2, 0)  (2, 3)   
4         (0, 1)  (1, 2)  (1, 1)  (3, 2)  (1, 0)  (0, 0)  (3, 1)  (2, 4)   
5         (0, 0)  (0, 2)  (1, 0)  (3, 1)  (0, 1)  (1, 1)  (2, 1)  (3, 3)   
6         (0, 1)  (0, 3)  (2, 0)  (3, 2)  (1, 0)  (1, 2)  (3, 0)  (4, 3)   
7         (1, 1)  (1, 2)  (1, 1)  (2, 1)  (1, 1)  (1, 1)  (4, 0)  (3, 4)   
8         (0, 1)  (2, 1)  (2, 1)  (2, 0)  (1, 0)  (2, 0)  (4, 1)  (2, 3)   
9         (0, 2)  (3, 1)  (1, 2)  (3, 1)  (0, 1)  (3, 0)  (4, 2)  (3, 3)   

                         
AgentID      9       10  
Step                     
0        (0, 1)  (1, 1)  
1        (0, 2)  (0, 1)  
2        (0, 3)  (0, 2)  
3        (1, 3)  (1, 1)  
4        (0, 3)  (2, 0)  
5        (1, 2)  (3, 1)  
6        (2, 1)  (2, 1)  
7        (2, 0)  (1, 1)  
8        (3, 1)  (1, 0)  
9        (3, 2)  (2, 0)